In [5]:
import pandas as pd    
import numpy as np
import scipy.stats as stats
import scipy
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [6]:
import urllib.request as request

**Imports all traffic data from 2007 - 2018**

Link: https://data.seattle.gov/browse?q=traffic%20flow%20count&sortBy=relevance to download yearly data

In [7]:
url_list = ['https://data-seattlecitygis.opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.csv',
            'https://data-seattlecitygis.opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.csv']
    

In [8]:
df_18 = pd.read_csv('https://data-seattlecitygis.opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.csv')

In [18]:
df_18 = df_18.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})

In [19]:
df_18

,OBJECTID,COMPKEY,STNAME,GEOBASID,DOWNTOWN,START_DATE,AMPK,PMPK,AWDT,ADT,AWDT_ROUNDED,DATAQUALITY,FLAGS,SHAPE_Length
0,1,12221,PINE ST,894,Y,1970/01/01 00:00:00+00,NaN,NaN,8000.0,NaN,8000,Estimate,NaN,322.037238
1,2,2203,15TH AVE W ON RP,1345,N,2015/03/13 00:00:00+00,NaN,NaN,11129.0,10139.0,11100,Study - 13-15,NaN,173.612269
2,3,"17275,17277,17276,17272,17273,17274,17271",NE 65TH ST,1622,N,2015/02/27 00:00:00+00,NaN,NaN,20487.0,19740.0,20500,Study - 13-15,NaN,1445.416389
3,4,"23302,23303,23304",W EMERSON ST,1371,N,2018/03/22 00:00:00+00,82.0,99.0,1007.0,1090.0,1000,Study(Normalized),NaN,999.190937
4,5,"19659,19656,19658,19655,19657,19654",S CLOVERDALE ST,126,N,2014/03/25 00:00:00+00,NaN,NaN,4868.0,4615.0,4900,Study - 13-15,NaN,2077.434844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1901,1919,NaN,LEARY AVE NW,1495,N,2014/01/09 00:00:00+00,NaN,NaN,17100.0,16081.0,17100,Study - 13-15,NaN,48.541264
1902,1920,NaN,15TH AVE NW,1496,N,2018/08/02 00:00:00+00,710.0,722.0,9970.0,9636.0,10000,Study(Normalized),NaN,484.633789
1903,1921,NaN,W DRAVUS ST,1294,N,1970/01/01 00:00:00+00,NaN,NaN,15000.0,NaN,15000,Estimate,NaN,160.422534
1904,1922,NaN,15TH AVE W,1987,N,2018/06/26 00:00:00+00,2356.0,3017.0,36897.0,35258.0,36000,Study(Normalized),NaN,1380.021208


In [26]:
def get_df(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    df_year = pd.read_csv(url_list[num])
    if year == 15 or year == 16:
        df_year = df_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
    elif year == 17 or year == 18:
        df_year = df_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 'STNAME_ORD' : 'STNAME'})
        
    df_year_AAWDT = df_year['AAWDT'].values
    df_year_geobase = df_year['GEOBASID'].values
    df_year_dist = df_year['SHAPE_Length'].values
    
    return df_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [21]:
df_15 = get_df(15)

In [22]:
df_15

,OBJECTID,COUNT_COMPKEY,STNAME,FIRST_ARTDESCRIPT,FIRST_TRANDESCRIPT,COUNTFLAG,GEOBASID,DOWNTOWN,COUNTSTART,COUNTAADT,AAWDT,ESTIMATE,AWDT_ROUND,SHAPE_Length
0,1,NaN,W SEA BR EB 4 AV OFF RP,NaN,NaN,Y,2028.0,N,NaN,NaN,24000.000000,Y,24000,1458.971527
1,2,NaN,MYERS WAY S,NaN,NaN,Y,2029.0,N,NaN,NaN,38000.000000,Y,38000,335.499087
2,3,NaN,S CLOVERDALE ST,NaN,NaN,Y,2030.0,N,NaN,NaN,13000.000000,Y,13000,987.546181
3,4,NaN,S CLOVERDALE ST,NaN,NaN,Y,2031.0,N,NaN,NaN,11000.000000,Y,11000,1305.323633
4,5,NaN,M L KING JR WAY S,NaN,NaN,Y,2032.0,N,NaN,NaN,12200.000000,Y,12200,609.897683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1855,1856,NaN,RENTON AVE S,NaN,NaN,Y,46.0,N,NaN,NaN,8700.000000,Y,8700,834.302025
1856,1857,NaN,35TH AVE SW/MARINE VIEW DR SW,NaN,NaN,Y,48.0,N,2014/02/19 00:00:00+00,1756.393489,1816.779275,NaN,1800,6728.714716
1857,1858,NaN,SW 106TH ST,NaN,NaN,Y,49.0,N,2015/05/05 00:00:00+00,13006.994000,13061.048000,NaN,13100,1250.039037
1858,1859,NaN,39TH AVE SW,NaN,NaN,Y,50.0,N,NaN,NaN,1500.000000,Y,1500,681.446100


In [17]:
montlake_br_traffic_07 = df_07.loc[df_07['STNAME']=='MONTLAKE BR']

In [19]:
montlake_br_traffic_07

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length
197,198,86850010,N,11902,MONTLAKE BR,83,"MONTLAKE BR, S/O POINT A",2007,11902,57203,0,,Y,1362.081215


In [49]:
df_048 = get_df(8)

In [11]:
def get_montlake_br_data(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    df_year = get_df(year)
    montlake_br_traffic_year = df_year.loc[df_year['STNAME'] == 'MONTLAKE BLVD NE']
    return montlake_br_traffic_year

In [26]:
montlake_br_data_08 = get_montlake_br_data(8)

In [27]:
montlake_br_data_08

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length
198,199,86850010,N,11902,MONTLAKE BR,165,"MONTLAKE BR, S/O POINT A",2008,11902,57100,0,,Y,1362.081215


In [28]:
montlake_br_data_by_year = pd.DataFrame()
year_list = range(7,19)
for year in year_list:
    num = year-7
    print(year)
    new = get_montlake_br_data(year)
    montlake_br_data_by_year = montlake_br_data_by_year.append(new)

7
8
9
10
11
12
13
14
15
16
17
18


In [29]:
montlake_br_data_by_year

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,...,COUNTAADT,ESTIMATE,AWDT_ROUND,START_DATE,AMPK,PMPK,ADT,AWDT_ROUNDED,DATAQUALITY,FLAGS
148,149,86950030.0,N,11911.0,MONTLAKE BLVD NE,188.0,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2007.0,11911.0,46000.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,150,86950030.0,N,11911.0,MONTLAKE BLVD NE,164.0,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2008.0,11911.0,46100.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,150,86950030.0,N,11911.0,MONTLAKE BLVD NE,152.0,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2009.0,11911.0,47800.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,150,86950030.0,N,11911.0,MONTLAKE BLVD NE,152.0,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2010.0,11911.0,45900.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,127,86950030.0,N,11911.0,MONTLAKE BLVD NE,NaN,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",NaN,11911.0,46800.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,109,86950030.0,N,11911.0,MONTLAKE BLVD NE,NaN,NaN,NaN,NaN,42312.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,156,86950030.0,N,11911.0,MONTLAKE BLVD NE,NaN,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2013.0,11911.0,44600.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,158,86950030.0,N,11911.0,MONTLAKE BLVD NE,NaN,"MONTLAKE BLVD NE, N/O NE PACIFIC PL",2014.0,11911.0,48400.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,562,1412.0,N,NaN,MONTLAKE BLVD NE,NaN,NaN,NaN,NaN,43594.803,...,38366.664,NaN,43600.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572,573,1423.0,N,NaN,MONTLAKE BLVD NE,NaN,NaN,NaN,NaN,31500.000,...,NaN,Y,31500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
